In [1]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}
org = set(teams.values())

In [210]:
import requests
import re
import pandas as pd
import pickle
from datetime import time, timedelta

In [4]:
def pbp_to_tdelt(time):
    if ':' in time:
        elem = pd.to_numeric(time.split(':'))
        time = pd.to_timedelta(elem[0],unit='m') + pd.to_timedelta(elem[1],unit='s')
        return time
    else:
        return time

In [347]:
def neg_check(strng,idx):
    elem = strng.split('-')
    if elem[idx] == '':
        del(elem[idx])
        elem[idx] = '-'+elem[idx]
    return elem

In [263]:
def get_table(res,code,tid):
    import re
    import pandas as pd
    
    comm = re.compile('<!--|-->')
    try:
        table = pd.read_html(comm.sub("",res.text),attrs={'id':tid},flavor='bs4')
    except ValueError:
        print(f'Problem with {tid} table in game {code}. Please check!')
        return None
    else:
        return table[0]

In [386]:
year = '2006'
path = 'gamecodes0.data'
with open(path, 'rb') as f:
    gcodes = pickle.load(f)
    
path = 'df_step1_'+year+'.data'
with open(path, 'rb') as f:
    dfyear = pickle.load(f)

In [387]:
ginfo = pd.DataFrame(columns=['Code','Surface','Temperature','Over/Under'])
ginfo['Code'] = gcodes
ginfo = ginfo.set_index(['Code'])

In [388]:
gc = [gcodes[i] for i in range(len(gcodes)) 
      if (((gcodes[i][0:4] == year) & (gcodes[i][4:6] != '01')) | (gcodes[i][0:6] == (str(int(year)+1)+'01')))]

In [389]:
len(gc)

256

In [ ]:
tableids = ['game_info','team_stats','player_defense','returns','kicking','home_drives','vis_drives','pbp']
for gamecode in gc:
    wpath = 'raw/'+year+'/'+gamecode+'.pkl'
    f = open(wpath, 'wb')
    url = 'https://www.pro-football-reference.com/boxscores/'+gamecode+'.htm'
    res = requests.get(url)
    table = pd.read_html(res.text,attrs={'class':'linescore nohover stats_table no_freeze'},flavor='bs4')
    pickle.dump(table[0],f)
    for tableid in tableids:
        table = get_table(res,gamecode,tableid)
        pickle.dump(table,f)
    f.close()
print('End')

In [385]:
for gamecode in gc:
    wpath = 'raw/'+year+'/'+gamecode+'.pkl'
    f = open(wpath, 'rb')
## Team Designation
    table = pickle.load(f)
    awayteam = table.iloc[0,1]
    away = teams[awayteam]
    hometeam = table.iloc[1,1]
    home = teams[hometeam]
# Points
    dfyear.loc[(away,gamecode),'Points'] = int(table.loc[0,'Final'])
    dfyear.loc[(home,gamecode),'Points'] = int(table.loc[1,'Final'])
    dfyear.loc[(away,gamecode),'Points_Opp'] = int(table.loc[1,'Final'])
    dfyear.loc[(home,gamecode),'Points_Opp'] = int(table.loc[0,'Final'])

# Game Info
    table = pickle.load(f)
    table = table.set_index(table.columns[0])
    ginfo.loc[gamecode,'Surface'] = table.loc['Surface',1]
    ginfo.loc[gamecode,'Over/Under'] = float(table.loc['Over/Under',1].split(' ')[0])
    if 'Weather' in table.index: ginfo.loc[gamecode,'Temperature'] = int(table.loc['Weather',1].split(' ')[0])
    table = pickle.load(f)
# Offensive & Defensive Rushing Yards
    rush1 = neg_check(table.iloc[1,1],1) # Check for negative rushing yards
    dfyear.loc[(away,gamecode),'Yds_Off_Rush'] = int(rush1[1])
    dfyear.loc[(home,gamecode),'Yds_Def_Rush'] = int(rush1[1])
    rush2 = neg_check(table.iloc[1,2],1) # Check for negative rushing yards
    dfyear.loc[(home,gamecode),'Yds_Off_Rush'] = int(rush2[1])
    dfyear.loc[(away,gamecode),'Yds_Def_Rush'] = int(rush2[1])
# Offensive & Defensive Passing Yards
    pass1 = neg_check(table.iloc[2,1],2) # Check for negative passing yards
    dfyear.loc[(away,gamecode),'Yds_Off_Pass'] = int(pass1[2])
    dfyear.loc[(home,gamecode),'Yds_Def_Pass'] = int(pass1[2])
    pass2 = neg_check(table.iloc[2,2],2) # Check for negative passing yards
    dfyear.loc[(home,gamecode),'Yds_Off_Pass'] = int(pass2[2])
    dfyear.loc[(away,gamecode),'Yds_Def_Pass'] = int(pass2[2])
# Time of Possession  (Converted to fractional minutes)  
    dfyear.loc[(away,gamecode),'Possession'] = \
        float(table.iloc[11,1].split(':')[0])+float(table.iloc[11,1].split(':')[1])/60.0
    dfyear.loc[(home,gamecode),'Possession'] = \
        float(table.iloc[11,2].split(':')[0])+float(table.iloc[11,2].split(':')[1])/60.0
# Penalties
    dfyear.loc[(away,gamecode),'Yds_Pen'] = int(table.iloc[8,1].split('-')[1])
    dfyear.loc[(home,gamecode),'Yds_Pen'] = int(table.iloc[8,2].split('-')[1])
# Defensive Sacks
    dfyear.loc[(away,gamecode),'Sacks_Def'] = int(table.iloc[3,2].split('-')[0])
    dfyear.loc[(home,gamecode),'Sacks_Def'] = int(table.iloc[3,1].split('-')[0])
# Turnovers
    dfyear.loc[(away,gamecode),'TO_Gained'] = int(table.iloc[7,2])
    dfyear.loc[(home,gamecode),'TO_Gained'] = int(table.iloc[7,1])
    dfyear.loc[(away,gamecode),'TO_Lost'] = int(table.iloc[7,1])
    dfyear.loc[(home,gamecode),'TO_Lost'] = int(table.iloc[7,2])
# Touchdowns (Defensive & Special Teams to be added later)
    dfyear.loc[(away,gamecode),'TD'] = int(rush1[2])+int(pass1[3])
    dfyear.loc[(home,gamecode),'TD'] = int(rush2[2])+int(pass2[3])
    
# Defense
    table = pickle.load(f)
# Remove multilevel, NANs, separator row
    table = table.droplevel(level=0,axis=1)
    table = table.dropna(subset=['Tm']).iloc[:,1:]
    table = table[table.iloc[:,0]!='Tm']

# Rename columns to remove column name duplication
    cols = list(table.columns)
    cols[3] = 'TD_Int'
    cols[2] = 'Yds_Int'
    cols[13] = 'Yds_Fum'
    cols[14] = 'TD_Fum'
    table.columns = cols

# Change value types to numeric types
    cols = [list(table.columns[1:6]),table.columns[6],list(table.columns[7:])]
    for grp in cols:
        table[grp]=table[grp].apply(pd.to_numeric)
        
# Group by team to calculate team values        
    grouped = table.groupby(table['Tm'],sort=False)
    
# Add Defensive Touchdowns
    dftd=grouped['TD_Int'].sum()+grouped['TD_Fum'].sum()
    dfyear.loc[(away,gamecode),'TD'] += dftd[0]
    dfyear.loc[(home,gamecode),'TD'] += dftd[1]
    dfyear.loc[(away,gamecode),'TD_on_Def'] = dftd[0]
    dfyear.loc[(home,gamecode),'TD_on_Def'] = dftd[1]
    
# Tackles for Loss
    tfl=grouped['TFL'].sum()
    dfyear.loc[(away,gamecode),'Tackles_Loss'] = tfl[0]
    dfyear.loc[(home,gamecode),'Tackles_Loss'] = tfl[1]

# Kick/Punt Returns
    table = pickle.load(f)
    if table is not None:
        table = table.droplevel(level=0,axis=1)
        table = table.dropna(subset=['Tm']).iloc[:,1:]
        table = table[table.iloc[:,0]!='Tm']
        cols = list(table.columns)
        cols[2] = 'Yds_KR'
        cols[7] = 'Yds_PR'
        cols[5] = 'Lng_KR'
    cols[10] = 'Lng_PR'
    cols[4] = 'TD_KR'
    cols[9] = 'TD_PR'
    table.columns = cols
    cols = [list(table.columns[1:3]),table.columns[3],list(table.columns[4:8]),table.columns[8],
            list(table.columns[9:])]
    for grp in cols:
        table[grp]=table[grp].apply(pd.to_numeric)

    grouped = table.groupby(table['Tm'],sort=False)
    kptd=grouped['TD_KR'].sum()+grouped['TD_PR'].sum()
    kyds=grouped['Yds_KR'].sum()
    knum=grouped['Rt'].sum()
    pyds=grouped['Yds_PR'].sum()
    pnum=grouped['Ret'].sum()
    if (kptd.shape[0] == 0):
        a_kptd = h_kptd = a_knum = h_knum = a_kyds = h_kyds = a_pyds = h_pyds = a_pnum = h_pnum = 0
    elif (kptd.shape[0] == 1):
        if (away.upper() == table.loc[0,'Tm']):
            a_kptd = kptd[0]
            a_knum = knum[0]
            a_kyds = kyds[0]
            a_pyds = pyds[0]
            a_pnum = pnum[0]
            h_kptd = h_knum = h_kyds = h_pyds = h_pnum = 0
        elif (home.upper() == table.loc[0,'Tm']):
            h_kptd = kptd[0]
            h_knum = knum[0]
            h_kyds = kyds[0]
            h_pyds = pyds[0]
            h_pnum = pnum[0]
            a_kptd = a_knum = a_kyds = a_pyds = a_pnum = 0
    else:
            a_kptd = kptd[0]
            a_knum = knum[0]
            a_kyds = kyds[0]
            a_pyds = pyds[0]
            a_pnum = pnum[0]
            h_kptd = kptd[1]
            h_knum = knum[1]
            h_kyds = kyds[1]
            h_pyds = pyds[1]
            h_pnum = pnum[1]
    dfyear.loc[(away,gamecode),'TD'] += a_kptd
    dfyear.loc[(home,gamecode),'TD'] += h_kptd
    if (a_knum > 0):
        dfyear.loc[(away,gamecode),'Yds_per_Kickret'] = float(a_kyds)/float(a_knum)
    else:
        dfyear.loc[(away,gamecode),'Yds_per_Kickret'] = 0.0
    if (h_knum > 0):
        dfyear.loc[(home,gamecode),'Yds_per_Kickret'] = float(h_kyds)/float(h_knum)
    else:
        dfyear.loc[(home,gamecode),'Yds_per_Kickret'] = 0.0
    if (a_pnum > 0):
        dfyear.loc[(away,gamecode),'Yds_per_Puntret'] = float(a_pyds)/float(a_pnum)
    else:
        dfyear.loc[(away,gamecode),'Yds_per_Puntret'] = 0.0
    if (h_pnum > 0):
        dfyear.loc[(home,gamecode),'Yds_per_Puntret'] = float(h_pyds)/float(h_pnum)
    else:
        dfyear.loc[(home,gamecode),'Yds_per_Puntret'] = 0.0

# Field Goals
    table = pickle.load(f)
    table = table.droplevel(level=0,axis=1)
    table = table.dropna(subset=['Tm']).iloc[:,1:]
    table = table[table.iloc[:,0]!='Tm']
    cols = [list(table.columns[1:7]),table.columns[7],table.columns[8]]
    for grp in cols:
        table[grp]=table[grp].apply(pd.to_numeric)
    grouped = table.groupby(table['Tm'],sort=False)
    fgm=grouped['FGM'].sum()
    fga=grouped['FGA'].sum()
    dfyear.loc[(away,gamecode),'FG_Made'] = fgm[0]
    dfyear.loc[(home,gamecode),'FG_Made'] = fgm[1]
    dfyear.loc[(away,gamecode),'FG_Att'] = fga[0]
    dfyear.loc[(home,gamecode),'FG_Att'] = fga[1]
# Total Plays & Set up Drive and Play-by-Play tables for Red Zone calculation
    table = pickle.load(f)
    hdrive = table.iloc[:,1:]
    hdrive.index = range(1,len(hdrive)+1)
    hdrive.index.set_names('Drive',inplace=True)
    hdrive.rename({'Net_yds':'Yds'},axis=1,inplace=True)
    dfyear.loc[(home,gamecode),'Plays'] = hdrive['Plays'].sum()
    table = pickle.load(f)
    adrive = table.iloc[:,1:]
    adrive.index = range(1,len(adrive)+1)
    adrive.index.set_names('Drive',inplace=True)
    adrive.rename({'Net_yds':'Yds'},axis=1,inplace=True)
    dfyear.loc[(away,gamecode),'Plays'] = adrive['Plays'].sum()

    hdrive['Quarter']=hdrive['Quarter'].fillna(method='ffill')
    adrive['Quarter']=adrive['Quarter'].fillna(method='ffill')
    hdrive['LOS']=hdrive['LOS'].fillna(home.upper()+' 25')
    position=hdrive['LOS'].str.split(expand=True)
    hdrive.drop('LOS',axis=1,inplace=True)
    hdrive[['Side','Marker']] = position
    adrive['LOS']=adrive['LOS'].fillna(away.upper()+' 25')
    position=adrive['LOS'].str.split(expand=True)
    adrive.drop('LOS',axis=1,inplace=True)
    adrive[['Side','Marker']] = position
    cols=['Quarter','Time','Side','Marker','Plays','Length','Yds','Result']
    hdrive[hdrive['Side']=='50']=hdrive[hdrive['Side']=='50'].assign(Marker = '50') # Case where ball is on 50 yard line
    hdrive[hdrive['Side']=='50']=hdrive[hdrive['Side']=='50'].assign(Side = home.upper())
    hdrive=hdrive.reindex(columns=cols,copy=False)
    adrive[adrive['Side']=='50']=adrive[adrive['Side']=='50'].assign(Marker = '50') # Case where ball is on 50 yard line
    adrive[adrive['Side']=='50']=adrive[adrive['Side']=='50'].assign(Side = home.upper())
    adrive=adrive.reindex(columns=cols,copy=False)

    intcols=['Plays','Yds','Quarter','Marker']
    timecols=['Time','Length']
    strcols=['Result','Side']
    for colu in intcols:
        hdrive[colu] = hdrive[colu].astype('int')
        adrive[colu] = adrive[colu].astype('int')
    for colu in strcols:
        hdrive[colu] = hdrive[colu].astype('str')
        adrive[colu] = adrive[colu].astype('str')
    for colu in timecols:
        for drive in range(1,hdrive.shape[0]+1):
            entry = hdrive.loc[drive,colu].split(':')
            hdrive.loc[drive,colu] = timedelta(hours=0,minutes=int(entry[0]),seconds=int(entry[1]))
        for drive in range(1,adrive.shape[0]+1):
            entry = adrive.loc[drive,colu].split(':')
            adrive.loc[drive,colu] = timedelta(hours=0,minutes=int(entry[0]),seconds=int(entry[1]))

    table = pickle.load(f)
    f.close()
    pbp=table.iloc[:,0:6]
    if (pbp.columns.nlevels > 1):                 # Sometimes multilevel object is retrieved. Ensure that
        pbp.columns = pbp.columns.droplevel()     # columns are single level
    pbp=pbp.dropna()
    pbp=pbp[pbp['Time'].str.contains(pat=':',regex=False)]
    pbp['Time'] = pbp['Time'].apply(pbp_to_tdelt)
    position=pbp['Location'].str.split(expand=True)
    position=position.iloc[:,0:2]
    pbp=pbp.drop('Location',axis=1)
    pbp[['Team','Location']] = position
    cols=['Quarter','Time','Down','ToGo','Team','Location','Detail']
    pbp=pbp.reindex(columns=cols,copy=False)
    pbp=pbp.reset_index(drop=True)
    pbp[pbp['Team']=='50']=pbp[pbp['Team']=='50'].assign(Location = '50') # Case where ball is on 50 yard line
    pbp[pbp['Team']=='50']=pbp[pbp['Team']=='50'].assign(Team = home.upper())
    pbp['Location']=pbp['Location'].astype('int')    
    pbp[pbp['Quarter']=='OT']=pbp[pbp['Quarter']=='OT'].assign(Quarter = '5') # Case of Overtime
    pbp['Quarter']=pbp['Quarter'].astype('int')

# Red Zone Routine

    team = {'H':home, 'A':away}
    drive = {home:1, away:1}
    df = {home:hdrive, away:adrive}
    rz_conv = {home:0, away:0}
    rz_att = {home:0, away:0}
    row = 0

#            Red Zone starts at 20 yard line
    while ((drive[home] <= hdrive.shape[0]) | (drive[away] <= adrive.shape[0])):
        stop = False
        if ((drive[home] <= hdrive.shape[0]) & (drive[away] <= adrive.shape[0])):
            if (hdrive.loc[drive[home],'Quarter'] < adrive.loc[drive[away],'Quarter']):
                poss = 'H'
                starttime = hdrive.loc[drive[home],'Time']
            elif (adrive.loc[drive[away],'Quarter'] < hdrive.loc[drive[home],'Quarter']):
                poss = 'A'
                starttime = adrive.loc[drive[away],'Time']
            else:
                if (hdrive.loc[drive[home],'Time'] > adrive.loc[drive[away],'Time']):
                    poss = 'H'
                    starttime = hdrive.loc[drive[home],'Time']
                else:
                    poss = 'A'
                    starttime = adrive.loc[drive[away],'Time']
        else:
            if (drive[home] <= hdrive.shape[0]):
                poss = 'H'
                starttime = hdrive.loc[drive[home],'Time']
            else:
                poss = 'A'
                starttime = adrive.loc[drive[away],'Time']
        rz = False   # Not in Red Zone
        bquar = df[team[poss]].loc[drive[team[poss]],'Quarter'] # Beginning quarter of drive
        if (df[team[poss]].loc[drive[team[poss]],'Length'] <= starttime): # Set time for next drive
            endtime = starttime - df[team[poss]].loc[drive[team[poss]],'Length']
            equar = bquar                                            # Ending quarter of drive
        else:    
            endtime = (timedelta(hours=0,minutes=15,seconds=0) -  # Assuming 15 minute quarters
                (df[team[poss]].loc[drive[team[poss]],'Length'] - starttime))
            equar = bquar+1                                            # Ending quarter of drive

        if (df[team[poss]].loc[drive[team[poss]],'Side'] != team[poss].upper()):
            if (df[team[poss]].loc[drive[team[poss]],'Marker'] <= 20):
                rz = True
            else:
                start = df[team[poss]].loc[drive[team[poss]],'Marker'] - 20
        else:
            start = 100 - df[team[poss]].loc[drive[team[poss]],'Marker'] - 20 # Other side of the field
        if not (rz):    # If starting outside Red Zone
            end = start - df[team[poss]].loc[drive[team[poss]],'Yds']   # Calculate end of drive
            if end <= 20: # If drive ends within 20 yards of red zone to the goal line check if goes in RZ
                found = False     # Look for beginning of drive
                while (((row+1) < pbp.shape[0]) & (not (found))):
                    if (pbp.loc[row,'Quarter'] < bquar):
                        row += 1
                    elif (pbp.loc[row,'Time'] <= starttime):
                        found = True
                    else:
                        row += 1
                while (not(stop)):
                    if ((pbp.loc[row,'Team'] != team[poss].upper()) & (pbp.loc[row,'Location'] <= 20)):
                        rz = True
                        stop = True
                    elif (pbp.loc[row,'Quarter'] > equar):
                        stop = True
                    elif (row+1) == pbp.shape[0]:                         # Failsafe in case of bad data
                        stop = True
                    elif ((pbp.loc[row,'Quarter'] == equar) & (pbp.loc[row,'Time'] <= endtime)):
                        stop = True
                    else:
                        row += 1
        if (rz):
            rz_att[team[poss]] += 1 # Increment Red Zone Attempts
            if df[team[poss]].loc[drive[team[poss]],'Result'] in ['Touchdown']:
                rz_conv[team[poss]] += 1  # If score, increment Red Zone successes 
        drive[team[poss]] += 1  # Increment drive number for team in question

    dfyear.loc[(away,gamecode),'RZ_Att'] = rz_att[away]
    dfyear.loc[(home,gamecode),'RZ_Att'] = rz_att[home]
    dfyear.loc[(away,gamecode),'RZ_Conv'] = rz_conv[away]
    dfyear.loc[(home,gamecode),'RZ_Conv'] = rz_conv[home]
    dfyear.loc[(away,gamecode),'RZ_Def_Att'] = rz_att[home]
    dfyear.loc[(home,gamecode),'RZ_Def_Att'] = rz_att[away]
    dfyear.loc[(away,gamecode),'RZ_Def_Conv'] = rz_conv[home]
    dfyear.loc[(home,gamecode),'RZ_Def_Conv'] = rz_conv[away]

intcols=['Points','Points_Opp','Yds_Off_Pass','Yds_Off_Rush','Yds_Def_Pass','Yds_Def_Rush','TD','TD_on_Def',
         'FG_Made','FG_Att','RZ_Conv','RZ_Att','RZ_Def_Conv','RZ_Def_Att','Plays','TO_Gained','TO_Lost',
         'Yds_Pen','Tackles_Loss']
fltcols=['Possession','Sacks_Def','Yds_per_Kickret','Yds_per_Puntret']
strcols=['Home/Away', 'Opponent']
for colu in intcols: dfyear[colu] = dfyear[colu].astype('int64')
for colu in fltcols: dfyear[colu] = dfyear[colu].astype('float64')
for colu in strcols: dfyear[colu] = dfyear[colu].astype('str')
print('End')

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


End


In [378]:
table

In [197]:
with open('df_step2_2010.data','rb') as f:
    dfold=pickle.load(f)

In [370]:
dfyear.iloc[:,12:].describe()

,RZ_Conv,RZ_Att,RZ_Def_Conv,RZ_Def_Att,Possession,Plays,TO_Gained,TO_Lost,Yds_Pen,Sacks_Def,Tackles_Loss,Yds_per_Kickret,Yds_per_Puntret
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,1.835938,3.892578,1.835938,3.892578,30.195215,63.835938,1.552734,1.552734,44.554688,2.023438,4.576172,22.574043,8.332118
std,1.268479,1.690940,1.268479,1.690940,4.596588,8.446009,1.300742,1.300742,24.975074,1.703983,2.285421,6.829185,9.882820
min,0.000000,0.000000,0.000000,0.000000,14.750000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-8.000000
25%,1.000000,3.000000,1.000000,3.000000,27.287500,58.000000,1.000000,1.000000,25.000000,1.000000,3.000000,18.666667,1.666667
50%,2.000000,4.000000,2.000000,4.000000,30.325000,64.000000,1.000000,1.000000,40.500000,2.000000,4.000000,21.666667,6.583333
75%,3.000000,5.000000,3.000000,5.000000,33.254167,70.000000,2.000000,2.000000,60.000000,3.000000,6.000000,25.517857,11.000000
max,7.000000,9.000000,7.000000,9.000000,45.250000,86.000000,7.000000,7.000000,134.000000,9.000000,13.000000,60.000000,89.000000


In [219]:
dfyear[dfyear['TD_on_Def']==dfyear.TD_on_Def.max()].iloc[:,:12]

,,Home/Away,Opponent,Points,Points_Opp,Yds_Off_Pass,Yds_Off_Rush,Yds_Def_Pass,Yds_Def_Rush,TD,TD_on_Def,FG_Made,FG_Att
Team,Code,,,,,,,,,,,,
rav,201110020rav,H,New York Jets,34,17,163,112,119,38,4,3,2,2


In [368]:
dfyear.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 512 entries, ('sdg', '200809070sdg') to ('crd', '200812280crd')
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Home/Away        512 non-null    object 
 1   Opponent         512 non-null    object 
 2   Points           512 non-null    int64  
 3   Points_Opp       512 non-null    int64  
 4   Yds_Off_Pass     512 non-null    int64  
 5   Yds_Off_Rush     512 non-null    int64  
 6   Yds_Def_Pass     512 non-null    int64  
 7   Yds_Def_Rush     512 non-null    int64  
 8   TD               512 non-null    int64  
 9   TD_on_Def        512 non-null    int64  
 10  FG_Made          512 non-null    int64  
 11  FG_Att           512 non-null    int64  
 12  RZ_Conv          512 non-null    int64  
 13  RZ_Att           512 non-null    int64  
 14  RZ_Def_Conv      512 non-null    int64  
 15  RZ_Def_Att       512 non-null    int64  
 16  Possession       512

In [372]:
path = 'df_step2_'+year+'.pkl'
with open(path, 'wb') as f:
    pickle.dump(dfyear,f)

In [371]:
dfyear.sample(10)

,,Home/Away,Opponent,Points,Points_Opp,Yds_Off_Pass,Yds_Off_Rush,Yds_Def_Pass,Yds_Def_Rush,TD,TD_on_Def,...,RZ_Def_Att,Possession,Plays,TO_Gained,TO_Lost,Yds_Pen,Sacks_Def,Tackles_Loss,Yds_per_Kickret,Yds_per_Puntret
Team,Code,,,,,,,,,,,,,,,,,,,,,
ram,200812210ram,H,San Francisco 49ers,16,17,233,129,216,83,1,0,...,2,38.633333,82,4,1,45,4.0,5,25.000000,9.000000
phi,200810050phi,H,Washington Redskins,17,23,196,58,194,203,2,0,...,3,25.250000,49,0,0,33,1.0,2,19.400000,68.000000
nyg,200810190nyg,H,San Francisco 49ers,29,17,161,112,256,35,3,0,...,1,34.500000,66,3,0,80,6.0,11,21.800000,4.000000
crd,200811230crd,H,New York Giants,29,37,351,23,240,87,3,0,...,7,29.383333,71,0,2,29,1.0,3,29.000000,-2.500000
mia,200812140mia,H,San Francisco 49ers,14,9,156,98,233,112,2,0,...,4,21.783333,43,0,0,66,5.0,8,23.333333,17.333333
nor,200809140was,A,Washington Redskins,24,29,216,55,321,149,3,0,...,6,25.766667,55,1,3,55,2.0,4,23.250000,55.000000
clt,200812140clt,H,Detroit Lions,31,21,318,106,233,90,4,0,...,3,32.683333,68,2,2,31,0.0,6,21.400000,10.000000
cin,200809210nyg,A,New York Giants,23,26,286,102,289,117,2,0,...,5,33.250000,75,0,0,57,0.0,3,19.000000,15.500000
phi,200812280phi,H,Dallas Cowboys,44,6,175,137,233,87,5,2,...,4,30.916667,62,5,1,0,4.0,7,11.000000,12.000000


In [129]:
path = 'df_step2_2015.data'
with open(path, 'rb') as f:
    dfy=pickle.load(f)

In [132]:
dfy.loc[:,'FG_Made':'FG_Att']

FG_Made  FG_Att
Team Code                         
atl  201509140atl        1       2
     201509200nyg        1       1
     201509270dal        1       1
     201510040atl        0       1
     201510110atl        2       3
...                    ...     ...
was  201512070was        3       4
     201512130chi        0       1
     201512200was        0       0
     201512260phi        1       1
     201601030dal        0       0

[512 rows x 2 columns]

In [133]:
dfy.loc[('atl','201510110atl')]

Home/Away                            H
Opponent           Washington Redskins
Points                              25
Points_Opp                          19
Yds_Off_Pass                       254
Yds_Off_Rush                       176
Yds_Def_Pass                       219
Yds_Def_Rush                        51
TD                                   1
TD_on_Def                            0
FG_Made                              2
FG_Att                               3
RZ_Conv                              2
RZ_Att                               5
RZ_Def_Conv                          2
RZ_Def_Att                           3
Possession                     34.9333
Plays                               81
TO_Gained                            2
TO_Lost                              3
Yds_Pen                             62
Sacks_Def                            1
Tackles_Loss                         3
Yds_per_Kickret                   26.5
Yds_per_Puntret                     10
Name: (atl, 201510110atl)